In [1]:
# Importing libraries and packages

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.utils import shuffle
import os
import re

from permutation_test_util import *

In [2]:
%reload_ext autoreload

In [3]:
# Adjust dataframe display options to view full output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 10)

In [4]:
# Importing dataset

os.chdir("C:/Users/phuro/UCSD/Project_WHEL/data")

fp = os.path.join('interim', 'fdr_correction_demo_dataset.csv')
df = pd.read_csv(fp, index_col=0)

In [5]:
quanti_cols = ["age at dx", "main meals", "snacks", "meals out", "cooking time", "adults", \
    "children", "shopping freq", "grocery bill", "weight", "time at weight",
    "max weight", "years in state", "# first deg relatives with breast cancer",
    "# second deg relatives with breast cancer", "# sisters", 
    "# sisters with breast cancer",	"# sisters with ovarian cancer",
    "# first deg relatives with colorectal cancer", 
    "# second deg relatives with colorectal cancer", "Menarch Age", 
    "# Times Pregnant", "# Live Births", "Age at First Live Birth", 
    "Months Full Breast Feeding", "Months Partial Breastfeeding", 
    "Age at Menopause", "Years of Birth Control", "Years of HRT",	
    "# Previous Breast Biopsies", "lutein", "cryptox", "lycopene", "alphacar", 
    "betacar", "BMI", "Bld Pres -Sys", "Bld Pres - Dias", "% Protein", "% Fat", 
    "% Carb", "% Alcohol", "pack years", "NEW METS", "Number of Supplements"] 
    

In [6]:
df.drop(columns='ID', inplace=True)
df

,age at dx,Ethnicity,Education,Marital Status,Empl. Status,site,status,main meals,snacks,meals out,cooking time,adults,children,I cook?,I clean up?,shopping freq,grocery bill,I buy food?,weight,time at weight,max weight,insurance change?,insurance satisfaction,years in state,where born,age15 residence,age35 residence,age50 residence,longest residence,adopted?,# first deg relatives with breast cancer,# second deg relatives with breast cancer,mom had BC?,mom had ovarian cancer?,# sisters,# sisters with breast cancer,# sisters with ovarian cancer,# first deg relatives with colorectal cancer,# second deg relatives with colorectal cancer,Menarch Age,# Times Pregnant,# Live Births,Age at First Live Birth,Months Full Breast Feeding,Months Partial Breastfeeding,Age at Menopause,Hysterectomy,# Oophorectomies,Hormonal Birth Control,Birth Control Type,Years of Birth Control,HRT,HRT Type,Years of HRT,# Previous Breast Biopsies,lutein,cryptox,lycopene,alphacar,betacar,Lump/Mast,Chemo,Tumor Type,BMI,Bld Pres -Sys,Bld Pres - Dias,% Protein,% Fat,% Carb,% Alcohol,smoking status,pack years,NEW METS,wgt history,hospital_,Blood_Sugar,Cardiovascular,Digestive,Arthritis,Osteoporosis,Number of Supplements,Amino Acids,Antioxidants,Carotenoids,Herbals/Not Phytoestrogens,Herbals/Phytoestrogens,Minerals/Calcium,Minerals/Not Calcium,Miscellaneous,Multi-Vitamin,Multi-vitamin/mineral,Other nutrients,Probiotics/Enzymes,Protein Supplements/Soy,Vitamin A,Vitamin B,Vitamin C,Vitamin D,Vitamin E,Vitamin K,vitality,othcstatus,grade,newstage
0,54,1,7,5,0,CA,unreturned,16.0,8.0,4.0,35.0,1.0,NaN,1.0,1.0,1.0,25.0,1.0,146.0,0.5,148.0,1.0,9.0,4.0,1,1.0,1.0,1.0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,10.0,2.0,2.0,18.0,6.0,3.0,51.0,0.0,0.0,1.0,1.0,5.0,1.0,2.0,10.0,NaN,0.5150,0.2428,0.9518,0.3087,1.0113,2,2,3,24.9108,124,96,18.924077,29.524726,50.142715,3.8873367,2,7.0000,945.0,4,0.0,0.0,0.0,1.0,0.0,0.0,4,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,2.0,1,2
1,57,1,9,2,0,AZ,returned,21.0,0.0,2.0,55.0,2.0,0.0,1.0,1.0,1.0,100.0,1.0,218.0,8.0,218.0,1.0,8.0,1.0,3,3.0,4.0,4.0,4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13.0,2.0,2.0,27.0,0.0,0.0,47.0,0.0,0.0,1.0,1.0,14.0,1.0,2.0,10.0,NaN,0.2237,0.1666,0.3300,0.1132,0.2663,2,2,3,37.8260,96,75,17.659007,24.663251,57.057056,3.4782122,1,22.5000,1785.0,2,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0.0,3,1
2,45,3,8,2,1,CA,unreturned,14.0,16.0,7.0,35.0,2.0,0.0,1.0,1.0,1.0,75.0,1.0,198.0,0.5,207.0,1.0,9.0,4.0,1,1.0,1.0,1.0,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,2.0,15.0,0.0,0.0,44.0,0.0,0.0,1.0,1.0,5.0,0.0,NaN,NaN,NaN,0.1574,0.1460,0.3600,0.0595,0.1581,1,2,3,34.6098,-9,-9,15.294056,31.070694,56.07326,0.0053804,3,0.0000,850.0,2,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,3,2
3,52,1,7,2,0,CA,unreturned,16.0,13.0,1.0,55.0,2.0,0.0,1.0,1.0,1.0,NaN,1.0,169.0,2.0,169.0,5.0,8.0,4.0,3,3.0,3.0,1.0,3,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,13.0,3.0,3.0,20.0,6.0,0.0,42.0,1.0,1.0,1.0,1.0,7.0,1.0,4.0,10.0,NaN,0.4145,0.1256,0.5800,0.2100,0.6182,1,1,3,29.5006,-9,-9,18.706712,32.766762,49.53105,0.1372782,2,25.0000,450.0,2,0.0,0.0,0.0,0.0,1.0,1.0,9,0,0,0,3,0,1,2,1,0,0,0,0,0,0,1,1,0,0,0,1,0.0,3,1
4,48,1,9,2,1,CA,returned,20.0,5.0,0.0,35.0,2.0,0.0,1.0,1.0,1.0,100.0,1.0,130.0,1.0,142.0,1.0,9.0,4.0,4,4.0,1.0,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,12.0,3.0,2.0,24.0,11.0,0.0,48.0,0.0,0.0,1.0,1.0,12.0,0.0,1.0,1.0,NaN,0.3378,0.3948,1.2372,0.5279,1.6302,1,2,3,22.6540,-9,-9,19.750513,30.613405,52.198382,0.0093277,3,0.0000,450.0,4,NaN,NaN,NaN,NaN,NaN,NaN,4,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [7]:
# -9 represents unknown entries, so replace -9 with NaN

df.replace(-9, np.nan, inplace=True)

In [8]:
# Get colors

red = px.colors.qualitative.Alphabet[17]
blue = px.colors.qualitative.Dark24[9]

In [9]:
# More EDA to determine multicollinearity and insignificant columns

meals_fig = px.bar(df, x='meals out', y='cooking time', \
    title='# meals eat out and cooking time', color_discrete_sequence=[blue])
meals_fig.update_layout(xaxis_title='# meals eat out, per week', \
    yaxis_title='cooking time, minutes/day')
meals_fig

In [10]:
shop_fig = px.histogram(df, color='I buy food?', x='shopping freq', \
    title='shopping frequency and whether patient buys groceries', color_discrete_sequence=[blue, red],
    opacity=0.75, log_y=True)
shop_fig.update_layout(xaxis_title='I buy groceries?', \
    yaxis_title='grocery shopping freq, per week')
shop_fig

In [11]:
# Drop columns "I buy food?", time at weight, max weight

df.drop(columns=['I buy food?', 'time at weight', 'max weight'], inplace=True)

In [12]:
# Plot a heat map of df

# plt.figure(figsize=(55, 31))
# ax = sns.heatmap(df.corr(), annot=True, fmt='.2f', linewidths=0.5)
# ticklabels = ax.xaxis.get_majorticklabels()
# props = plt.setp(ticklabels, rotation=45, ha='right', rotation_mode='anchor')
# plt.title('Correlation heatmap', size=15)

In [13]:
# Get highly-correlated relationships 

cor = df.corr() 

for var1 in cor.columns:
    for var2 in cor.index:
        if var1 == var2:
            continue
        val = cor.loc[var2, var1]
        if abs(val) > 0.7:
            print(f'{var2} and {var1}: {val}')
        

age50 residence and age at dx: -0.7570630600729278
BMI and weight: 0.9214077849908399
age at dx and age50 residence: -0.7570630600729278
mom had BC? and # first deg relatives with breast cancer: 0.7396872425690922
# first deg relatives with breast cancer and mom had BC?: 0.7396872425690922
# Live Births and # Times Pregnant: 0.8134514185485883
# Times Pregnant and # Live Births: 0.8134514185485883
weight and BMI: 0.9214077849908399
Herbals/Not Phytoestrogens and Number of Supplements: 0.7246538188393399
Number of Supplements and Herbals/Not Phytoestrogens: 0.7246538188393399


In [14]:
# Drop columns weight, # Live Births, mom had BC?, Herbals/Not Phytoestrogens

df.drop(columns=['weight', '# Live Births', 'mom had BC?', 'Herbals/Not Phytoestrogens'], inplace=True)

In [86]:
# Check empty values 

df.isna().sum().value_counts()

0      36
11      7
481     6
2       5
40      5
       ..
15      1
20      1
475     1
14      1
10      1
Length: 33, dtype: int64

In [121]:
# Convert data to correct types

df = df.replace(r'^\s+$', np.NaN, regex=True)

for col in df.dtypes.index:
    if col in quanti_cols:
        df[col] = df[col].astype(float)

In [122]:
df.dtypes

age at dx         float64
Ethnicity           int64
Education           int64
Marital Status      int64
Empl. Status        int64
                   ...   
Vitamin K           int64
vitality            int64
othcstatus        float64
grade               int64
newstage            int64
Length: 97, dtype: object

In [123]:
# Instantiate FDR Control object

fdr_control = FDRController()

In [139]:
# Run permutation tests

fdr_control.test(
    df, 
    5000, 
    shuffle_column='status',
    quantitative_columns=quanti_cols
)

In [140]:
# Get P-values

fdr_control.get_pvalues()

[0.0,
 0.0002,
 0.0162,
 0.1132,
 0.1512,
 0.0016,
 0.0,
 0.2004,
 0.002,
 0.5314,
 0.3174,
 0.0,
 0.5626,
 0.1092,
 0.7306,
 0.0108,
 0.6594,
 0.3628,
 0.0178,
 0.0046,
 0.0726,
 0.0002,
 0.0,
 0.0,
 0.5362,
 0.2982,
 0.6754,
 0.6862,
 0.023,
 0.2716,
 0.8592,
 0.0196,
 0.4458,
 0.9732,
 0.189,
 0.013,
 0.3716,
 0.0046,
 0.0,
 0.2282,
 0.2152,
 0.4366,
 0.8926,
 0.6418,
 0.0,
 0.009,
 0.9552,
 0.4728,
 0.0006,
 0.0042,
 0.0166,
 0.0096,
 0.001,
 0.0,
 0.0,
 0.2518,
 0.0004,
 0.1254,
 0.6064,
 0.5906,
 0.0386,
 0.0742,
 0.23,
 0.0332,
 0.0924,
 0.013,
 0.418,
 0.8718,
 0.6564,
 0.1874,
 0.6602,
 0.9088,
 0.1084,
 0.5928,
 0.5046,
 0.6876,
 0.8462,
 0.6518,
 0.0208,
 0.7516,
 0.2824,
 0.381,
 0.6168,
 0.7098,
 1.0,
 0.7668,
 0.3592,
 0.8986,
 0.7346,
 0.8776,
 0.0924,
 1.0,
 0.0,
 0.757,
 0.0002,
 0.0]

In [141]:
# Number of variables/attributes

fdr_control.k

96

In [142]:
# Adjust p-values (calculate qvalues)

fdr_control.adjust()

In [143]:
# Get Q-values

fdr_control.get_qvalues()

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.001476923076923077,
 0.0013714285714285716,
 0.0013714285714285716,
 0.00256,
 0.0036,
 0.005647058823529412,
 0.008533333333333334,
 0.010105263157894737,
 0.02016,
 0.020072727272727273,
 0.020072727272727273,
 0.03756521739130434,
 0.0384,
 0.041471999999999995,
 0.04622222222222222,
 0.04622222222222222,
 0.05495172413793103,
 0.05495172413793103,
 0.056960000000000004,
 0.06069677419354839,
 0.0624,
 0.06690909090909092,
 0.09374117647058823,
 0.10587428571428573,
 0.19251891891891892,
 0.19251891891891892,
 0.22744615384615385,
 0.22744615384615385,
 0.25568780487804876,
 0.25568780487804876,
 0.25874285714285716,
 0.27996279069767444,
 0.32989090909090907,
 0.39443478260869563,
 0.39443478260869563,
 0.40932765957446804,
 0.43039999999999995,
 0.44160000000000005,
 0.44160000000000005,
 0.47397647058823533,
 0.5014153846153846,
 0.5115169811320754,
 0.5301333333333333,
 0.5540072727272728,
 0.6110315789473685,


In [144]:
# Determine critical threshold to keep/exclude rejected null hypotheses at a 
# 5% false discovery rate
# default fdr is 5%

fdr_control.assess()

In [145]:
# Get critical threshold

fdr_control.get_threshold()

0.04622222222222222

In [146]:
# Get full results

fdr_control.get_results()

,feature,p-values,q-values
0,age at dx,0.0000,0.000000
92,vitality,0.0000,0.000000
54,Chemo,0.0000,0.000000
53,Lump/Mast,0.0000,0.000000
44,HRT,0.0000,0.000000
...,...,...,...
71,Arthritis,0.9088,0.948313
46,Years of HRT,0.9552,0.986013
33,Menarch Age,0.9732,0.993906
91,Vitamin K,1.0000,1.000000


In [147]:
# Get rejected null hypotheses (features) that are accounted for with 
# false discovery correction

to_reject = fdr_control.get_results(reject_only=True)
to_reject

,feature,p-values,q-values
0,age at dx,0.0000,0.000000
92,vitality,0.0000,0.000000
54,Chemo,0.0000,0.000000
53,Lump/Mast,0.0000,0.000000
44,HRT,0.0000,0.000000
...,...,...,...
45,HRT Type,0.0090,0.037565
51,alphacar,0.0096,0.038400
15,grocery bill,0.0108,0.041472
65,NEW METS,0.0130,0.046222
